![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2Farchitectures%2Ftracking%2Fsetup%2Fgithub&dt=GitHub+Metrics+-+2+-+Incremental+Update+Cloud+Function.ipynb)

# GitHub Metrics: Cloud Function For Incremental Updates

---
## COLAB SETUP

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/architectures/tracking/setup/github/GitHub%20Metrics%20-%202%20-%20Incremental%20Update%20Cloud%20Function.ipynb) and run the cells in this section.  Otherwise, skip this section.